In [17]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords


import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import datetime
import nltk
import nltk.data
from nltk.util import bigrams 
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
import collections
from collections import Counter
import re
import string
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings(action='ignore', category=FutureWarning, module='pyLDAvis')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet
from pprint import pprint
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim


In [2]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

/anaconda3/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

[nltk_data] Downloading package wordnet to /Users/varsha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [5]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varsha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [11]:

from sklearn import metrics
from nltk.stem.porter import PorterStemmer
article_hash = {}
directory = "/Users/varsha/GWU_SPRING2019/Topics in Big Data/Project/Code/Daily News"


files = [f for f in listdir(directory) if isfile(join(directory, f))]

for file in files:
    #file = pathlib.Path( "/Users/varsha/Python Workspace/homework_2/data")
    #print(file)
    fileop=open(directory+"/"+file,"r")
    text = fileop.read()
    article_hash[file]  = text
    print("-----------------------------------")
    print("file;" , file)
    #print(article_hash[file])

-----------------------------------
file; 2017-05-24.txt
-----------------------------------
file; 2017-05-30.txt
-----------------------------------
file; 2017-05-18.txt
-----------------------------------
file; 2017-02-11.txt
-----------------------------------
file; 2017-02-05.txt
-----------------------------------
file; 2017-12-28.txt
-----------------------------------
file; 2017-12-14.txt
-----------------------------------
file; 2017-10-05.txt
-----------------------------------
file; 2017-10-11.txt
-----------------------------------
file; 2017-07-21.txt
-----------------------------------
file; 2017-09-18.txt
-----------------------------------
file; 2017-09-30.txt
-----------------------------------
file; 2017-09-24.txt
-----------------------------------
file; 2017-07-09.txt
-----------------------------------
file; 2017-07-08.txt
-----------------------------------
file; 2017-09-25.txt
-----------------------------------
file; 2017-09-19.txt
-------------------------------

In [12]:
newspath = os.path.join(os.getcwd(), 'Daily News')

In [13]:
newspath

'/Users/varsha/GWU_SPRING2019/Topics in Big Data/Project/Code/Daily News'

In [14]:
nltk.download('wordnet')


newspath = os.path.join(os.getcwd(),'Daily News')



def get_topics(text):
    def lemmatize_stemming(text):
        stemmer = SnowballStemmer("english", ignore_stopwords=True)
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

    def preprocess(text):
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                #result.append(token)
                result.append(lemmatize_stemming(token))
        return result


    words = []
    for word in text.split(' '):
        words.append(word)

    processed_data = preprocess(text)

    dictionary = gensim.corpora.Dictionary([processed_data])

    bow_corpus = [dictionary.doc2bow(processed_data)]

    bow_doc_0 = bow_corpus[0]

    tfidf = models.TfidfModel(bow_corpus)

    corpus_tfidf = tfidf[bow_corpus]

    # LDA Model using Bag of Words
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=15, id2word=dictionary, passes=2, workers=4)

    topic_score_list = lda_model.show_topics(num_topics=1, num_words=15, log=False, formatted=False)[0][1]
    topics_list = [topic[0] for topic in topic_score_list]
    return(topics_list)
    




def compute_afinn_score(text):
    afinn = Afinn(emoticons=True)

    #compute sentiment scores (polarity) and labels
    sentiment_score     = [afinn.score(text)]
    sentiment_category  = ['positive' if score > 0
                           else 'negative' if score < 0
                           else 'neutral'
                           for score in sentiment_score]

    #print(sentiment_score)
    #print(sentiment_category)
    return(sentiment_score, sentiment_category)


def process_text(source,text):
    article_df = pd.DataFrame(columns=['source','topics'])      # initialize dataframe for each article
    #person_names=person_list

    #article_df['a'] = None

    #person_names = get_names(text)
    article_df['source'] = pd.Series(dtype='str')
    article_df['source'] = source
    
    topics = get_topics(text)
    article_df['topics'] = [topics]
    article_df['source'] = source
    (sentiment_score, sentiment_category) = compute_afinn_score(text)
    article_df['sentiment_score'] = sentiment_score
    article_df['sentiment_category'] = sentiment_category
    return article_df




results_df = pd.DataFrame()

[nltk_data] Downloading package wordnet to /Users/varsha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import numpy as np
from afinn import Afinn

newsfiles = [f for f in listdir(newspath) if isfile(join(newspath, f))]

for file in newsfiles:
   filepath = newspath + "/" + file
   data = open(filepath,'r',encoding='utf-8')
   text = data.read()
   results_df = results_df.append(process_text(file,text))

In [19]:




results_df['source'] = results_df['source'].astype(str)
print(results_df)



            source                                             topics  \
0   2017-05-24.txt  [share, profit, group, announc, trump, plan, m...   
0   2017-05-30.txt  [profit, loss, share, rise, million, post, ban...   
0   2017-05-18.txt  [share, rise, profit, trump, unit, announc, de...   
0   2017-02-11.txt  [trump, stake, report, profit, share, rise, fi...   
0   2017-02-05.txt  [trump, result, stand, preview, profit, china,...   
0   2017-12-28.txt  [say, announc, share, unit, hold, stock, china...   
0   2017-12-14.txt  [share, say, profit, unit, bank, deal, rate, a...   
0   2017-10-05.txt  [announc, share, bank, sale, stock, price, pla...   
0   2017-10-11.txt  [share, profit, sale, bank, offer, technolog, ...   
0   2017-07-21.txt  [share, profit, earn, bank, unit, million, eur...   
0   2017-09-18.txt  [announc, share, hold, agreement, group, unit,...   
0   2017-09-30.txt  [result, share, loss, million, announc, trump,...   
0   2017-09-24.txt  [result, stand, trump, zealand,

In [20]:

writer = pd.ExcelWriter('results_daily_news.xlsx')
results_df.to_excel(writer,'results')
writer.save()

exit()


